# Generate networks from empirical connectomes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv
from joblib import Parallel, delayed
import networkx as nx
import igraph as ig
import random
import graph_tool.all as gt
import subprocess

## Functions

In [ ]:
def from_edge_to_adj(edges): # Dataframe with edges

    edge1 = list(edges[0]-1)
    edge2 = list(edges[1]-1)

    N = np.max(edge1+edge2)+1

    adj_mat = np.zeros([N,N], dtype=int)

    for i in range(len(edge1)):
        adj_mat[edge1[i],edge2[i]] = 1
        adj_mat[edge2[i],edge1[i]] = 1
        
    return adj_mat

In [ ]:
def mean_degree_ig(G): # Compute the mean degree of a network 
    
    N = G.vcount()
    
    return np.sum(G.degree()) / N 

In [ ]:
def mean_degree_nx(G): # Compute the mean degree of a network 
    
    k = 0
    N = G.number_of_nodes()
    
    for i in range(N):
        k += G.degree[i]
        
    k_mean = k/N
    
    return k_mean

### Networks

In [ ]:
foldername = './Emp_connectomes'

filenames = os.listdir(foldername)
filenames.sort()

## Configuration Model

In [ ]:
def write_CM(filename):

    read_filename = foldername + '/' + filename

    read_edges = pd.read_csv(read_filename, sep=" ", header=None)
    read_edges = read_edges.drop(read_edges.columns[-1],axis=1)
    read_adj = from_edge_to_adj(read_edges)
    read_deg_list = list(read_adj.sum(axis=0))

    for i in range(20):
        
        if not os.path.exists('./CM_connectomes/' + str(i)):
            os.mkdir('./CM_connectomes/' + str(i))

        g_ig = ig.GraphBase.Degree_Sequence(read_deg_list, method = "vl")
        g_nx = nx.from_numpy_matrix( np.array(g_ig.get_adjacency()) )

        write_edges = list(g_nx.edges())

        write_filename = './CM_connectomes/' + str(i) + '/' + filename

        if os.path.exists(write_filename):
            os.remove(write_filename)

        for r in write_edges:
            with open(write_filename, "a") as f:
                writer = csv.writer(f, delimiter=' ')
                writer.writerow((r[0] + 1, r[1] + 1))

In [ ]:
#Parallel(n_jobs=-1, verbose=50)(delayed(write_CM)(file) for file in filenames)

In [ ]:
# Check networks
read_filename = './CM_connectomes/' + '0' + '/' + filenames[0]
read_edges = pd.read_csv(read_filename, sep=" ", header=None)
read_adj = from_edge_to_adj(read_edges)

g = nx.from_numpy_matrix(read_adj)
print('N:', g.number_of_nodes())
print('k:', mean_degree_nx(g))

## Erdos Renyi

In [ ]:
def write_ER(filename):

    read_filename = foldername + '/' + filename

    read_edges = pd.read_csv(read_filename, sep=" ", header=None)
    read_edges = read_edges.drop(read_edges.columns[-1],axis=1)
    read_adj = from_edge_to_adj(read_edges)
    g = nx.from_numpy_matrix(read_adj)
    
    N = g.number_of_nodes()
    k = mean_degree_nx(g)

    for i in range(20):
        
        if not os.path.exists('./ER_connectomes/' + str(i)):
            os.mkdir('./ER_connectomes/' + str(i))

        print(i)
        
        p = k / (N - 1)
        g_ER = nx.erdos_renyi_graph(N, p)

        print(mean_degree_nx(g_ER))

        write_edges = list(g_ER.edges())

        write_filename = './ER_connectomes/' + str(i) + '/' + filename

        if os.path.exists(write_filename):
            os.remove(write_filename)

        for r in write_edges:
            with open(write_filename, "a") as f:
                writer = csv.writer(f, delimiter=' ')
                writer.writerow((r[0] + 1, r[1] + 1))

In [ ]:
#Parallel(n_jobs=-1, verbose=50)(delayed(write_ER)(file) for file in filenames)

In [ ]:
# Check networks
read_filename = './ER_connectomes/' + '0' + '/' + filenames[0]
read_edges = pd.read_csv(read_filename, sep=" ", header=None)
read_adj = from_edge_to_adj(read_edges)

g = nx.from_numpy_matrix(read_adj)
print('N:', g.number_of_nodes())
print('k:', mean_degree_nx(g))

## Stochastic Block Model

In [ ]:
def block_pref_write(read_filename):
    
    read_edges = pd.read_csv(read_filename, sep=" ", header=None)
    read_edges = read_edges.drop(read_edges.columns[-1],axis=1)
    read_adj = from_edge_to_adj(read_edges)
    
    g = gt.Graph(directed=False)
    g.add_edge_list(np.transpose(read_adj.nonzero()))

    state = gt.minimize_blockmodel_dl(g)
    entr  = state.entropy()
    for i in range(1,2):
        state_n = gt.minimize_blockmodel_dl(g)
        entr_n  = state_n.entropy()
        if entr_n < entr:
            state = state_n
            entr  = entr_n

    e = state.get_matrix()
    e1 = pd.DataFrame(e.toarray())

    # Block sizes
    b = state.get_blocks()

    v = list()
    for e in b: v.append(e)

    d = dict()
    for e in v:
        if e not in d: d[e] = 1
        else:          d[e] = d[e] + 1

    mydict = dict()
    for key in sorted(d): mydict[key] = d[key]

    bs = list()
    for key in mydict: bs.append(mydict[key])

    df = pd.DataFrame(np.array(bs).reshape(1,len(bs)), columns=None)
    df.to_csv('block_sizes.csv', header=False, index=False)

    # Pref matrix
    e1 = e1/2
    for i in range(0,len(e1)):
        e1[i][i] = e1[i][i]/2
        
    tot_poss_edge = pd.DataFrame(np.zeros((len(e1),len(e1))))
    
    for i in range(0,len(tot_poss_edge)):
        tot_poss_edge[i][i] = (df[i][0]*(df[i][0]-1))/2
        
    for i in range(0, len(tot_poss_edge)):
        for j in range(0, len(tot_poss_edge)):
            if i != j:
                tot_poss_edge[i][j] = df[i][0]*df[j][0]
    
    pref = e1/tot_poss_edge
    pref.to_csv('pref_matrix.csv', header=False, index=False)

In [ ]:
def write_SBM(filename, inputpath, outputpath):
    
    read_filename = inputpath + '/' + filename
    
    block_pref_write(read_filename)
    
    subprocess.run(['Rscript', 'SBM_gen.R', outputpath  + '/' + filename])
    
    os.remove('block_sizes.csv')
    os.remove('pref_matrix.csv')

In [ ]:
for i in range(20):
        
    outputpath = './SBM_connectomes/' + str(i)
    
    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
    
    for filename in filenames:
        print(i, filename)
        write_SBM(filename, foldername, outputpath)

In [ ]:
# Check networks
read_filename = './SBM_connectomes/' + '0' + '/' + filenames[0]
read_edges = pd.read_csv(read_filename, sep=" ", header=None)
read_adj = from_edge_to_adj(read_edges)

g = nx.from_numpy_matrix(read_adj)
print('N:', g.number_of_nodes())
print('k:', mean_degree_nx(g))